move file (pd) to dedicated 

# In SQL

In [56]:
%reload_ext sql
%config SqlMagic.autocommit=False
%config SqlMagic.autolimit=0
%config SqlMagic.autopandas=True
%config SqlMagic.displaylimit=200

In [57]:
%sql trino://localhost:9090/cuebiq/

'Connected: @cuebiq/'

In [58]:
from sqlalchemy.engine import create_engine
output_schema_name = 'presence_data'
con = create_engine(f"trino://localhost:9090/dedicated/{output_schema_name}")

In [59]:
import time
import pandas as pd
from sqlalchemy import create_engine
from trino.dbapi import connect
from datetime import datetime, timedelta

In [60]:
%%sql
show schemas from dedicated

 * trino://localhost:9090/cuebiq/
Done.


,Schema
0,censustocensus
1,dataflowtest
2,default
3,information_schema
4,od_matrix
5,pop_density
6,presence_data
7,statetostatematrices
8,temporaryoutput


In [61]:
%%sql
show tables from dedicated.pop_density

 * trino://localhost:9090/cuebiq/
Done.


,Table
0,pd_co_20191004_agg3
1,pd_co_20191004_agg5
2,pd_co_20191005_agg3
3,pd_co_20191005_agg5
4,pd_co_20191006_agg3
...,...
583,pd_in_20190103_agg5
584,pd_in_20190104_agg3
585,pd_in_20190104_agg5
586,pd_in_20190105_agg3


In [86]:
%%sql
show tables from dedicated.od_matrix

 * trino://localhost:9090/cuebiq/
Done.


,Table
0,od_id_20190101_agg3
1,od_id_20190101_agg5
2,od_id_20190102_agg3
3,od_id_20190102_agg5
4,od_id_20190103_agg3
...,...
284,od_id_20190813_agg3
285,od_id_20190814_agg3
286,od_id_20190815_agg3
287,od_id_20190816_agg3


In [18]:
%%sql
show tables from dedicated.presence_data

 * trino://localhost:9090/cuebiq/
Done.


,Table
0,od_id_20190101_agg3
1,od_id_20190101_agg5
2,od_id_20190102_agg3
3,od_id_20190102_agg5
4,od_id_20190103_agg3
...,...
426,pd_id_20190729_agg5
427,pd_id_20190730_agg3
428,pd_id_20190730_agg5
429,pd_id_20190731_agg5


## Create schema

In [43]:
# output_schema_name = "OD_matrix"

In [44]:
# %sql create schema if not exists dedicated.$output_schema_name

 * trino://localhost:9090/cuebiq/
Done.


""


## Create table 

In [ ]:
# output_schema_name = 'presence_data'
# table_name = f"dedicated.{output_schema_name}.new_table_demo"

In [ ]:
# %sql create table if not exists $table_name (col_0 varchar, col_1 bigint, col_2 varchar)

In [10]:
# %sql show tables from dedicated.presence_data

## Delete table

In [88]:
%%sql
DROP TABLE IF EXISTS hive.od_matrix.od_id_20190301_agg5

 * trino://localhost:9090/cuebiq/
Done.


""


## Rename table

In [89]:
# Rename the table to the final name based on date and country code

output_schema_name = 'od_matrix'
con = create_engine(f"trino://localhost:9090/dedicated/{output_schema_name}")

rename_table_query = f"""
ALTER TABLE "od_id_20190301_test_agg5" RENAME TO "od_id_20190301_agg5"
"""

# Execute the rename table query
with con.connect() as connection:
    connection.execute(rename_table_query)

OperationalError: (trino.exceptions.TrinoExternalError) TrinoExternalError(type=EXTERNAL, name=HIVE_METASTORE_ERROR, message="New location for this table hive.od_matrix.od_id_20190301_agg5 already exists : s3a://cuebiq-paas-org-1872-kr7pteibwb/metastore/od_matrix.db/od_id_20190301_agg5", query_id=20240606_145713_00482_44guw)
[SQL: 
ALTER TABLE "od_id_20190301_test_agg5" RENAME TO "od_id_20190301_agg5"
]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

## Check table

In [44]:
# check
q2 = f"select * from dedicated.presence_data.od_id_20190301_agg5"

%sql $q2

 * trino://localhost:9090/cuebiq/
Done.


,start_geohash5,start_geohash_user,end_geohash5,end_geohash_user,trip_count,m_duration_min,mdn_duration_min,sd_duration_min,m_length_m,mdn_length_m,sd_length_m,partition_key
0,w0whv,221,w0wjj,112,9,65.850000,44.533333,57.140755,3980.066965,3976.456267,2251.684673,6
1,w0whv,221,w0wjt,31,1,35.050000,35.050000,NaN,12332.903097,12332.903097,NaN,6
2,w0whv,221,w0wk2,67,1,291.216667,291.216667,NaN,24480.067862,24480.067862,NaN,6
3,w0whw,295,w0whq,597,9,104.677778,42.950000,108.386946,7698.186495,4482.616607,7945.710727,6
4,w0whw,295,w0wjj,112,1,167.733333,167.733333,NaN,13154.413261,13154.413261,NaN,6
...,...,...,...,...,...,...,...,...,...,...,...,...
21785,qqtsv,58,qqtth,61,1,45.433333,45.433333,NaN,5461.877536,5461.877536,NaN,1
21786,qqtsw,97,qqts7,34,2,167.558333,167.558333,107.350594,28408.940774,28408.940774,23037.460306,1
21787,qqtsw,97,qqtss,40,1,64.483333,64.483333,NaN,10217.421722,10217.421722,NaN,1
21788,qqtsw,97,qqtsx,54,1,11.400000,11.400000,NaN,6957.994025,6957.994025,NaN,1


In [77]:
# check
q2 = f"select * from dedicated.od_matrix.od_id_20190303_agg5"
%sql $q2

 * trino://localhost:9090/cuebiq/
Done.


,start_geohash5,start_geohash_user,end_geohash5,end_geohash_user,trip_count,m_duration_min,mdn_duration_min,sd_duration_min,m_length_m,mdn_length_m,sd_length_m,partition_key
0,qqwdg,45,qqwe8,988,1,122.433333,122.433333,NaN,21584.564515,21584.564515,NaN,3
1,qqwdk,21,qqwdk,21,2,127.716667,127.716667,16.876282,80868.614001,80868.614001,20088.861007,3
2,qqwds,72,qqw7z,1068,1,63.566667,63.566667,NaN,33962.982120,33962.982120,NaN,3
3,qqwds,72,qqwdf,12,1,180.916667,180.916667,NaN,11544.041636,11544.041636,NaN,3
4,qqwds,72,qqwdu,50,3,55.761111,30.833333,53.979822,72520.508816,4646.784813,118370.595647,3
...,...,...,...,...,...,...,...,...,...,...,...,...
23097,qw8h9,31,qw8h6,69,1,17.166667,17.166667,NaN,6066.474576,6066.474576,NaN,4
23098,qw8h9,31,qw8nq,1003,1,270.516667,270.516667,NaN,42141.872608,42141.872608,NaN,4
23099,qw8he,15,qw81c,135,1,4.283333,4.283333,NaN,54372.636522,54372.636522,NaN,4
23100,qw8hf,23,qw8j4,88,4,138.887500,32.133333,218.480805,18706.251224,2985.613596,32669.492335,4


In [91]:
# check
q1 = f"select * from dedicated.pop_density.pd_id_20191009_agg3"
%sql $q1

 * trino://localhost:9090/cuebiq/


NameError: name 'DatabaseError' is not defined

In [ ]:
import pandas as pd
from sqlalchemy.engine import create_engine

# Replace with your actual schema name
output_schema_name = "presence_data"
bucketized_table = f"dedicated.{output_schema_name}.bucketized_table"

# Partition size
partition_size = 5000

# Data preparation
df['partition_key'] = (df.index // partition_size) + 1
df = df.astype({
    'start_geohash_user': 'int',
    'end_geohash_user': 'int',
    'trip_count': 'int',
    'partition_key': 'int'
})

# Create the SQL engine
con = create_engine(f"trino://localhost:9090/dedicated/{output_schema_name}")

# Define the SQL query to create the bucketized and partitioned table
create_table_query = f"""
CREATE TABLE {bucketized_table} (
    start_geohash5 varchar,
    start_geohash_user bigint,
    end_geohash5 varchar,
    end_geohash_user bigint,
    trip_count bigint,
    m_duration_min double,
    mdn_duration_min double,
    sd_duration_min double,
    m_length_m double,
    mdn_length_m double,
    sd_length_m double,
    partition_key bigint
)
WITH (
  partitioned_by = ARRAY['partition_key'],
  bucketed_by = ARRAY['end_geohash5'],
  bucket_count = 5
)
"""

# Execute the create table query
with con.connect() as connection:
    connection.execute(create_table_query)

# Function to insert data in chunks
def insert_data_in_chunks(df, table_name, engine, chunk_size=5000):
    for start in range(0, len(df), chunk_size):
        chunk = df.iloc[start:start + chunk_size]
        chunk.to_sql(table_name, engine, index=False, if_exists='append', method='multi')

# Insert data into the bucketized table
insert_data_in_chunks(df, "bucketized_table", con, chunk_size)

country_code = 'id'
date_str = "20190101"
final_bucketized_table = f"od_{country_code.lower()}_{event_date}_agg3"

# Rename the table to the final name based on the date range and country code
rename_table_query = f"""
ALTER TABLE "bucketized_table" RENAME TO "{final_bucketized_table}"
"""

# Execute the rename table query
with con.connect() as connection:
    connection.execute(rename_table_query)

## Read and write 

## From existing file (in jupyter)

In [6]:
# Generate date strings from 20190101 to 20190430
start_date = datetime.strptime("20190714", "%Y%m%d")
end_date = datetime.strptime("20190731", "%Y%m%d")
date_generated = [start_date + timedelta(days=x) for x in range(0, (end_date-start_date).days+1)]

# Function to insert data with retry mechanism
def insert_data_with_retry(df, table_name, con, retries=3, delay=5):
    for attempt in range(retries):
        try:
            df.to_sql(
                table_name, 
                con, 
                index=False, 
                if_exists="append", 
                method="multi"
            )
            print(f"Inserted data into table {table_name}")
            break
        except Exception as e:
            print(f"Attempt {attempt+1} failed with error: {e}")
            if attempt < retries - 1:
                time.sleep(delay)
            else:
                print(f"Failed to insert data into table {table_name} after {retries} attempts")

# Start timing the process
start_time = time.time()
                
# Iterate over each date, read the corresponding CSV file, and insert into the database for both agg3 and agg5
for date in date_generated:
    date_str = date.strftime("%Y%m%d")

    # Process agg5 files
    file_path_agg5 = f"/home/jovyan/Data/Agg_DL/ID5/{date_str}_ID_pe_dl_agg5.csv"
    table_name_agg5 = f"pd_id_{date_str}_agg5"
    
    try:
        # Read the CSV file for agg5
        df_agg5 = pd.read_csv(file_path_agg5)
        
        # Convert all column names to lowercase
        df_agg5.columns = [col.lower() for col in df_agg5.columns]
        
        # Insert DataFrame into the table with retry mechanism
        insert_data_with_retry(df_agg5, table_name_agg5, con)
    except Exception as e:
        print(f"Failed to process {file_path_agg5}: {e}")

    # Process agg3 files
    file_path_agg3 = f"/home/jovyan/Data/Agg_DL/ID3/{date_str}_ID_pe_dl_agg3.csv"
    table_name_agg3 = f"pd_id_{date_str}_agg3"
    
    try:
        # Read the CSV file for agg3
        df_agg3 = pd.read_csv(file_path_agg3)
        
        # Convert all column names to lowercase
        df_agg3.columns = [col.lower() for col in df_agg3.columns]
        
        # Insert DataFrame into the table with retry mechanism
        insert_data_with_retry(df_agg3, table_name_agg3, con)
    except Exception as e:
        print(f"Failed to process {file_path_agg3}: {e}")
        
# End timing the process
end_time = time.time()
total_time = end_time - start_time

print(f"Total time taken: {total_time} seconds")

# 638s for 20190511 - 20190713

Inserted data into table pd_id_20190714_agg5
Inserted data into table pd_id_20190714_agg3
Inserted data into table pd_id_20190715_agg5
Inserted data into table pd_id_20190715_agg3
Inserted data into table pd_id_20190716_agg5
Inserted data into table pd_id_20190716_agg3
Inserted data into table pd_id_20190717_agg5
Inserted data into table pd_id_20190717_agg3
Inserted data into table pd_id_20190718_agg5
Inserted data into table pd_id_20190718_agg3
Inserted data into table pd_id_20190719_agg5
Inserted data into table pd_id_20190719_agg3
Inserted data into table pd_id_20190720_agg5
Inserted data into table pd_id_20190720_agg3
Inserted data into table pd_id_20190721_agg5
Inserted data into table pd_id_20190721_agg3
Inserted data into table pd_id_20190722_agg5
Inserted data into table pd_id_20190722_agg3
Inserted data into table pd_id_20190723_agg5
Inserted data into table pd_id_20190723_agg3
Inserted data into table pd_id_20190724_agg5
Inserted data into table pd_id_20190724_agg3
Inserted d

In [19]:
# Single one for testing

# # Generate date strings from 20190101 to 20190430
# start_date = datetime.strptime("20190101", "%Y%m%d")
# end_date = datetime.strptime("20190104", "%Y%m%d")
# date_generated = [start_date + timedelta(days=x) for x in range(0, (end_date-start_date).days+1)]

# # Iterate over each date, read the corresponding CSV file, and insert into the database
# for date in date_generated:
#     date_str = date.strftime("%Y%m%d")
#     file_path = f"/home/jovyan/Data/Agg_DL/CO5/{date_str}_CO_pe_dl_agg5.csv"
#     table_name = f"pd_co_{date_str}_agg5"
    
#     try:
#         # Read the CSV file
#         df = pd.read_csv(file_path)
        
#         # Convert all column names to lowercase
#         df.columns = [col.lower() for col in df.columns]
        
#         # Insert DataFrame into the table
#         df.to_sql(
#             table_name, 
#             con, 
#             index=False, 
#             if_exists="append", 
#             method="multi"
#         )
#         print(f"Inserted data into table {table_name} from {file_path}")
#     except Exception as e:
#         print(f"Failed to process {file_path}: {e}")

# CO 5 不行 不知道为啥？？？ 


Failed to process /home/jovyan/Data/Agg_DL/CO5/20190101_CO_pe_dl_agg5.csv: error 502: bad gateway


Error closing cursor
Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/sqlalchemy/engine/base.py", line 1900, in _execute_context
    self.dialect.do_execute(
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/trino/sqlalchemy/dialect.py", line 365, in do_execute
    cursor.execute(statement, parameters)
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/trino/dbapi.py", line 482, in execute
    self._prepare_statement(operation, statement_name)
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/trino/dbapi.py", line 379, in _prepare_statement
    query.execute()
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/trino/client.py", line 807, in execute
    status = self._request.process(response)
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/trino/client.py", line 617, in process
    self.raise_response_error(http_response)
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/tri

Failed to process /home/jovyan/Data/Agg_DL/CO5/20190102_CO_pe_dl_agg5.csv: error 502: bad gateway


Error closing cursor
Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/sqlalchemy/engine/base.py", line 1900, in _execute_context
    self.dialect.do_execute(
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/trino/sqlalchemy/dialect.py", line 365, in do_execute
    cursor.execute(statement, parameters)
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/trino/dbapi.py", line 482, in execute
    self._prepare_statement(operation, statement_name)
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/trino/dbapi.py", line 379, in _prepare_statement
    query.execute()
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/trino/client.py", line 807, in execute
    status = self._request.process(response)
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/trino/client.py", line 617, in process
    self.raise_response_error(http_response)
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/tri

Failed to process /home/jovyan/Data/Agg_DL/CO5/20190103_CO_pe_dl_agg5.csv: error 502: bad gateway


Error closing cursor
Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/sqlalchemy/engine/base.py", line 1900, in _execute_context
    self.dialect.do_execute(
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/trino/sqlalchemy/dialect.py", line 365, in do_execute
    cursor.execute(statement, parameters)
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/trino/dbapi.py", line 482, in execute
    self._prepare_statement(operation, statement_name)
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/trino/dbapi.py", line 379, in _prepare_statement
    query.execute()
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/trino/client.py", line 807, in execute
    status = self._request.process(response)
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/trino/client.py", line 617, in process
    self.raise_response_error(http_response)
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/tri

Failed to process /home/jovyan/Data/Agg_DL/CO5/20190104_CO_pe_dl_agg5.csv: error 502: bad gateway


### test 

In [10]:
# import pandas as pd
# df = pd.read_csv('/home/jovyan/Data/Agg_DL/ID3/20190101_ID_pe_dl_agg3.csv')
# df

,geohash_3,point_number,user_number
0,qpx,1255,43
1,qpz,46864,845
2,qqe,1309,48
3,qqg,1981239,22776
4,qqs,57836,1189
...,...,...,...
132,wb2,34656,622
133,wb3,1331,54
134,wb4,3246,87
135,wb7,670,28


In [18]:
# %%time
# df.to_sql(
#     "my_processed_dataset3", 
#     con, 
#     index=False, 
#     if_exists="append", 
#     method="multi"
# )

CPU times: user 121 ms, sys: 0 ns, total: 121 ms
Wall time: 2.37 s


137

1. if_exists='fail': The method will raise a ValueError if the table already exists. This is the default behavior.
2. if_exists='replace': If the table exists, it will be dropped and replaced with the new data.
3. if_exists='append': If the table exists, the new data will be inserted into the existing table. If the table does not exist, it will be created.

 * trino://localhost:9090/cuebiq/
Done.


,geohash_5,point_number,user_number
0,t9qxx,1681,32
1,t9qxz,1708,54
2,t9qyq,344,12
3,t9qyr,657,18
4,t9qyt,957,18
...,...,...,...
10947,whguy,408,21
10948,whgwq,497,14
10949,whu5b,645,15
10950,whujm,381,12


# test

In [ ]:
df = pd.read_csv('/home/jovyan/Data/DL/MX/20190128_MX_pe_dl.csv')
# df.sort_values('event_datetime_local')
df

In [3]:
import gc
import os
import time
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime, timedelta

In [4]:
# SQL engine
from trino.dbapi import connect 
from sqlalchemy import create_engine
import pandas as pd
import time

class TrinoEngine():
    def __init__(self):
        conn = connect(
            host="localhost",
            port=9090,
            catalog="cuebiq"
        )
        self.cur = conn.cursor()
        self.engine = create_engine("trino://localhost:9090/cuebiq/")
    
    def execute_statement(self, query:str) -> list:
        """
        Create and drop statements.
        """
        self.cur.execute(query)
        return self.cur.fetchall()
    
    def read_sql(self, query:str) -> pd.DataFrame: 
        """
        Select and insert into operations.
        """
        return pd.read_sql(query, self.engine)

sql_engine = TrinoEngine()

In [5]:
schema_name = {'cda': 'cuebiq.paas_cda_pe_v3'}

# dl_table = f"{schema_name['cda']}.device_location"  
pe_dl_table = f"{schema_name['cda']}.device_location_uplevelled"

tj_table = f"{schema_name['cda']}.trajectory"     
pe_tj_table = f"{schema_name['cda']}.trajectory_uplevelled"

# stop_table = f"{schema['cda']}.stop" 
pe_stop_table = f"{schema_name['cda']}.stop_uplevelled"

visit_table = f"{schema_name['cda']}.visit " 

In [6]:
class TrinoEngine:
    def __init__(self):
        self.conn = connect(
            host="localhost",
            port=9090,
            catalog="cuebiq"
        )
        self.cur = self.conn.cursor()
        self.engine = create_engine("trino://localhost:9090/cuebiq/")
    
    def execute_statement(self, query: str):
        self.cur.execute(query)
        return self.cur.fetchall()
    
    def read_sql_chunked(self, query: str, chunksize: int = 10000):
        return pd.read_sql(query, self.engine, chunksize=chunksize)

sql_engine = TrinoEngine()


In [39]:
country_code = 'ID'
start_date = 20190101
end_date = 20190103

# Convert integer dates to datetime objects
start_date_dt = datetime.strptime(str(start_date), '%Y%m%d')
end_date_dt = datetime.strptime(str(end_date), '%Y%m%d')
formatted_date = start_date_dt.strftime('%Y%m%d')
next_date = (start_date_dt + timedelta(days=1)).strftime('%Y-%m-%d')

In [42]:
pe_tj_table = sql_engine.read_sql_chunked(
    f"""
    SELECT 
        cuebiq_id, 
        event_zoned_datetime, 
        processing_date,
        lat,
        lng, 
        TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s')) AS event_datetime_local
    FROM {pe_dl_table}
    WHERE 
        processing_date = {formatted_date} 
        AND country_code = '{country_code}' 
        AND event_zoned_datetime IS NOT NULL
        AND TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s')) IS NOT NULL
        AND TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s')) >= date_parse('{start_date_dt.strftime('%Y%m%d')}', '%Y%m%d')
        AND TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s')) <= date_parse('{next_date}', '%Y-%m-%d')
    """
)

# pe_tj_table
first_chunk = next(pe_tj_table)
first_chunk

,cuebiq_id,event_zoned_datetime,processing_date,lat,lng,event_datetime_local
0,1514830569,2019-01-01T20:09:18+07:00,20190101,-7.322733,108.222739,2019-01-01 20:09:18
1,1514830569,2019-01-01T20:05:16+07:00,20190101,-7.319290,108.220144,2019-01-01 20:05:16
2,1514830569,2019-01-01T20:13:38+07:00,20190101,-7.319290,108.220144,2019-01-01 20:13:38
3,1514830569,2019-01-01T20:17:39+07:00,20190101,-7.312219,108.211619,2019-01-01 20:17:39
4,1514830569,2019-01-01T19:59:15+07:00,20190101,-7.306623,108.206429,2019-01-01 19:59:15
...,...,...,...,...,...,...
9995,1513028254,2019-01-01T08:22:51+07:00,20190101,-6.525583,106.854246,2019-01-01 08:22:51
9996,1513028254,2019-01-01T05:32:23+07:00,20190101,-6.525322,106.854464,2019-01-01 05:32:23
9997,1513028254,2019-01-01T05:32:23+07:00,20190101,-6.525322,106.854464,2019-01-01 05:32:23
9998,1513028254,2019-01-01T05:32:24+07:00,20190101,-6.525322,106.854464,2019-01-01 05:32:24


In [31]:
pe_tj_table = sql_engine.read_sql_chunked(
    f"""
    SELECT 
        cuebiq_id, 
        event_zoned_datetime, 
        processing_date,
        lat,
        lng, 
        TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s')) AS event_datetime_local,
        DATE(TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s'))) AS event_date_local

    FROM {pe_dl_table}
    WHERE 
        processing_date = {formatted_date} 
        AND country_code = '{country_code}' 
        AND event_zoned_datetime IS NOT NULL
        AND TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s')) IS NOT NULL
        AND TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s')) >= date_parse('{start_date_dt.strftime('%Y%m%d')}', '%Y%m%d')
        AND TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s')) <= date_parse('{next_date}', '%Y-%m-%d')
    """
)

# Convert the generator to a DataFrame
chunks = [chunk for chunk in pe_tj_table]
pe_tj_table_df = pd.concat(chunks, ignore_index=True)

pe_tj_table_df

# Process date 用前面的

ProgrammingError: (trino.exceptions.TrinoUserError) TrinoUserError(type=USER_ERROR, name=SYNTAX_ERROR, message="line 11:10: mismatched input '<'. Expecting: '(', 'LATERAL', 'TABLE', 'UNNEST', <identifier>", query_id=20240604_181830_00347_tt5ya)
[SQL: 
    SELECT 
        cuebiq_id, 
        event_zoned_datetime, 
        processing_date,
        lat,
        lng, 
        TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s')) AS event_datetime_local,
        DATE(TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s'))) AS event_date_local

    FROM <generator object SQLDatabase._query_iterator at 0x7fecf51b2430>
    WHERE 
        processing_date = 20190101 
        AND country_code = 'IN' 
        AND event_zoned_datetime IS NOT NULL
        AND TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s')) IS NOT NULL
        AND TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s')) >= date_parse('20190101', '%Y%m%d'

In [16]:
# probabliy not correct!!!!
# 因为要求时间<开始时间

from datetime import datetime, timedelta
import pandas as pd

import time

# Start timing
start_time = time.time()

# Define the input parameters
country_code = 'ID'
start_date = 20190101
end_date = 20190103

# Convert integer dates to datetime objects
start_date_dt = datetime.strptime(str(start_date), '%Y%m%d')
end_date_dt = datetime.strptime(str(end_date), '%Y%m%d')

# Calculate the date 45 days before the start_date
lookback_date_dt = start_date_dt - timedelta(days=45)
formatted_lookback_date = lookback_date_dt.strftime('%Y%m%d')
formatted_end_date = end_date_dt.strftime('%Y%m%d')

# Construct the SQL query
pe_tj_table = sql_engine.read_sql_chunked(
    f"""
    SELECT 
        cuebiq_id, 
        event_zoned_datetime, 
        processing_date,
        lat,
        lng, 
        TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s')) AS event_datetime_local,
        DATE(TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s'))) AS event_date_local

    FROM {pe_dl_table}
    WHERE 
        processing_date >= {formatted_lookback_date}
        AND processing_date <= {formatted_end_date}
        AND country_code = '{country_code}' 
        AND event_zoned_datetime IS NOT NULL
        AND TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s')) IS NOT NULL
        AND TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s')) >= date_parse('{start_date_dt.strftime('%Y%m%d')}', '%Y%m%d')
        AND TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s')) <= date_parse('{end_date_dt.strftime('%Y-%m-%d')}', '%Y-%m-%d')
    """
)

# Convert the generator to a DataFrame
chunks = [chunk for chunk in pe_tj_table]
pe_tj_table_df = pd.concat(chunks, ignore_index=True)


# End timing
end_time = time.time()

# Calculate and print the total time taken
total_time = end_time - start_time
print(f"Total time taken: {total_time} seconds")

pe_tj_table_df

# probabliy not correct!!!!

KeyboardInterrupt: 

In [7]:
# This one seems like correct 
from datetime import datetime, timedelta
import pandas as pd
import time

# Start timing
start_time = time.time()

# Define the input parameters
country_code = 'MX'
start_date = 20190102
end_date = 20190103

# Convert integer dates to datetime objects
start_date_dt = datetime.strptime(str(start_date), '%Y%m%d')
end_date_dt = datetime.strptime(str(end_date), '%Y%m%d')

# Loop through each day from start_date to end_date
current_date = start_date_dt
while current_date <= end_date_dt:
    # Calculate the lookback and lookahead dates
    lookback_date = current_date - timedelta(days=1)
    lookahead_date = current_date + timedelta(days=35)
    
    # Format dates for the SQL query
    formatted_lookback_date = lookback_date.strftime('%Y%m%d')
    formatted_current_date = current_date.strftime('%Y%m%d')
    formatted_lookahead_date = lookahead_date.strftime('%Y%m%d')
    
    # Construct and execute the SQL query
    pe_dl_table = sql_engine.read_sql_chunked(
        f"""
        SELECT 
            cuebiq_id, 
            event_zoned_datetime, 
            processing_date,
            lat,
            lng, 
            TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s')) AS event_datetime_local,
            DATE(TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s'))) AS event_date_local

        FROM {pe_dl_table}
        WHERE 
            processing_date >= {formatted_lookback_date} 
            AND processing_date <= {formatted_lookahead_date}
            AND country_code = '{country_code}' 
            AND event_zoned_datetime IS NOT NULL
            AND TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s')) IS NOT NULL
            AND DATE(TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s'))) = date_parse('{formatted_current_date}', '%Y%m%d')
        """
    )
    
    # Convert the generator to a DataFrame
    chunks = [chunk for chunk in pe_tj_table]
    if chunks:
        pe_tj_table_df = pd.concat(chunks, ignore_index=True)
        
        # Save the DataFrame to a CSV file
        output_filename = f'/home/jovyan/Data/Test/0604/pe_tj_table_{formatted_current_date}.csv'
        pe_tj_table_df.to_csv(output_filename, index=False)
        print(f"Saved data for {formatted_current_date} to {output_filename}")
    
    # Move to the next day
    current_date += timedelta(days=1)

print("Data extraction and saving completed.")


end_time = time.time() # End timing

# Calculate and print the total time taken
total_time = end_time - start_time
print(f"Total time taken: {total_time} seconds")


Saved data for 20190102 to /home/jovyan/Data/Test/0604/pe_tj_table_20190102.csv
Saved data for 20190103 to /home/jovyan/Data/Test/0604/pe_tj_table_20190103.csv
Data extraction and saving completed.
Total time taken: 1972.0941424369812 seconds


In [8]:
# transform all event date and get one day. 
import time

country_code = 'MX'
start_date = 20190101

# Convert integer dates to datetime objects
start_date_dt = datetime.strptime(str(start_date), '%Y%m%d')
next_date = (start_date_dt + timedelta(days=1)).strftime('%Y-%m-%d')

# Start timing
start_time = time.time()

pe_tj_table = sql_engine.read_sql_chunked(
    f"""
    WITH event_data AS (
        SELECT 
            cuebiq_id, 
            event_zoned_datetime, 
            processing_date,
            timezoneoffset_secs,
            lat,
            lng, 
            TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s')) AS event_datetime_local
        FROM {pe_dl_table}
        WHERE 
            country_code = '{country_code}' 
            AND event_zoned_datetime IS NOT NULL
            AND TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s')) IS NOT NULL
    )
    SELECT 
        cuebiq_id, 
        event_zoned_datetime, 
        processing_date,
        timezoneoffset_secs,
        lat,
        lng
    FROM event_data
    WHERE 
        event_datetime_local >= date_parse('{start_date_dt.strftime('%Y%m%d')}', '%Y%m%d')
        AND event_datetime_local < date_parse('{next_date}', '%Y-%m-%d')
    """
)

# Convert the generator to a DataFrame
chunks = [chunk for chunk in pe_tj_table]
pe_tj_table_df = pd.concat(chunks, ignore_index=True)

# End timing
end_time = time.time()

# Calculate and print the total time taken
total_time = end_time - start_time
print(f"Total time taken: {total_time} seconds")


pe_tj_table_df


# transform all event date and get one day. 
# 877 for one day in ID 
# 1980 for one day in MX

Total time taken: 1980.333106994629 seconds


,cuebiq_id,event_zoned_datetime,processing_date,timezoneoffset_secs,lat,lng
0,1698771180,2019-01-01T07:54:30-06:00,20190101,-21600,17.992812,-93.373950
1,1698771180,2019-01-01T15:56:44-06:00,20190101,-21600,18.000913,-93.371119
2,1698770637,2019-01-01T15:14:05-06:00,20190101,-21600,22.761220,-102.537551
3,1698770637,2019-01-01T02:11:25-06:00,20190101,-21600,22.761236,-102.537586
4,1698770637,2019-01-01T01:03:56-06:00,20190101,-21600,22.761512,-102.537348
...,...,...,...,...,...,...
20410846,1133006951,2019-01-01T16:18:02-06:00,20190127,-21600,24.035794,-104.652039
20410847,1133006951,2019-01-01T16:38:48-06:00,20190127,-21600,24.036156,-104.652051
20410848,1133006951,2019-01-01T17:06:00-06:00,20190127,-21600,24.036156,-104.652051
20410849,1133006951,2019-01-01T18:08:46-06:00,20190127,-21600,24.036297,-104.651470


In [11]:
pip install geohash2

  Using cached geohash2-1.1-py3-none-any.whl
  Using cached docutils-0.21.2-py3-none-any.whl (587 kB)
Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install python-geohash

Note: you may need to restart the kernel to use updated packages.


In [5]:
# with logging and works. export data to the system. 
# 1933.101960659027 for 3 day in IN
# process date -1 to +35
from datetime import datetime, timedelta
import pandas as pd
import time
import geohash2
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

start_time = time.time()  # Start timing

# Define the input parameters
country_code = 'IN'
start_date = 20190101
end_date = 20190103

# Convert integer dates to datetime objects
start_date_dt = datetime.strptime(str(start_date), '%Y%m%d')
end_date_dt = datetime.strptime(str(end_date), '%Y%m%d')

# Loop through each day from start_date to end_date
current_date = start_date_dt
while current_date <= end_date_dt:
    # Calculate the lookback and lookahead dates
    lookback_date = current_date - timedelta(days=1)
    lookahead_date = current_date + timedelta(days=35)
    
    # Format dates for the SQL query
    formatted_lookback_date = lookback_date.strftime('%Y%m%d')
    formatted_current_date = current_date.strftime('%Y%m%d')
    formatted_lookahead_date = lookahead_date.strftime('%Y%m%d')
    
    # Construct the SQL query
    query = f"""
    SELECT 
        cuebiq_id, 
        event_zoned_datetime, 
        processing_date,
        lat,
        lng
    FROM {pe_dl_table}
    WHERE 
        processing_date BETWEEN {formatted_lookback_date} AND {formatted_lookahead_date}
        AND country_code = '{country_code}' 
        AND event_zoned_datetime IS NOT NULL
        AND TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s')) IS NOT NULL
        AND DATE(TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s'))) = date_parse('{formatted_current_date}', '%Y%m%d')
    """
    
    logging.info(f"Executing SQL query for date {formatted_current_date}: {query}")
    
    try:
        pe_dl_table_gen = sql_engine.read_sql_chunked(query)
        
        # Convert the generator to a DataFrame
        chunks = [chunk for chunk in pe_dl_table_gen]
        if chunks:
            pe_dl_table_df = pd.concat(chunks, ignore_index=True)
            
            # Calculate geohashes
            pe_dl_table_df['geohash5'] = pe_dl_table_df.apply(lambda row: geohash2.encode(row['lat'], row['lng'], precision=5), axis=1)
            pe_dl_table_df['geohash3'] = pe_dl_table_df.apply(lambda row: geohash2.encode(row['lat'], row['lng'], precision=3), axis=1)
            
            # Aggregate data for geohash5
            aggregated_data_5 = pe_dl_table_df.groupby('geohash5').agg(
                no_of_points=('geohash5', 'size'),
                no_of_unique_users=('cuebiq_id', 'nunique')
            ).reset_index()
            
            # Save the aggregated data to a CSV file for geohash5
            output_filename_5 = f'/home/jovyan/Data/Agg_DL/IN5/aggregated_pe_tj_table_geohash5_{formatted_current_date}.csv'
            aggregated_data_5.to_csv(output_filename_5, index=False)
            logging.info(f"Saved aggregated data for geohash5 for {formatted_current_date} to {output_filename_5}")

            # Aggregate data for geohash3
            aggregated_data_3 = pe_dl_table_df.groupby('geohash3').agg(
                no_of_points=('geohash3', 'size'),
                no_of_unique_users=('cuebiq_id', 'nunique')
            ).reset_index()
            
            # Save the aggregated data to a CSV file for geohash3
            output_filename_3 = f'/home/jovyan/Data/Agg_DL/IN3/aggregated_pe_tj_table_geohash3_{formatted_current_date}.csv'
            aggregated_data_3.to_csv(output_filename_3, index=False)
            logging.info(f"Saved aggregated data for geohash3 for {formatted_current_date} to {output_filename_3}")
        
    except Exception as e:
        logging.error(f"Error while processing data for date {formatted_current_date}: {e}")
    
    # Move to the next day
    current_date += timedelta(days=1)

logging.info("Data extraction, aggregation, and saving completed.")

end_time = time.time()  # End timing

# Calculate and print the total time taken
total_time = end_time - start_time
logging.info(f"Total time taken: {total_time} seconds")


2024-06-04 18:47:51,936 - INFO - Executing SQL query for date 20190101: 
    SELECT 
        cuebiq_id, 
        event_zoned_datetime, 
        processing_date,
        lat,
        lng
    FROM cuebiq.paas_cda_pe_v3.device_location_uplevelled
    WHERE 
        processing_date BETWEEN 20181231 AND 20190205
        AND country_code = 'IN' 
        AND event_zoned_datetime IS NOT NULL
        AND TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s')) IS NOT NULL
        AND DATE(TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s'))) = date_parse('20190101', '%Y%m%d')
    
2024-06-04 18:57:53,341 - INFO - Saved aggregated data for geohash5 for 20190101 to /home/jovyan/Data/Agg_DL/IN3/aggregated_pe_tj_table_geohash5_20190101.csv
2024-06-04 18:57:55,233 - INFO - Saved aggregated data for geohash3 for 20190101 to /home/jovyan/Data/Agg_DL/IN5/aggregated_pe_tj_table_geohash3_20190101.csv
2024-06-04 18:57:55,234 - INFO - Executing SQL query for date 20190

In [7]:
# Write in to the table

from datetime import datetime, timedelta
import pandas as pd
import time
import geohash2
import logging
from sqlalchemy import create_engine

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Database connection setup
output_schema_name = 'presence_data'
con = create_engine(f"trino://localhost:9090/dedicated/{output_schema_name}")

start_time = time.time()  # Start timing

# Define the input parameters
country_code = 'IN'
start_date = 20190101
end_date = 20190103

# Define the input schema and table name
schema_name = {'cda': 'cuebiq.paas_cda_pe_v3'}
pe_dl_table = f"{schema_name['cda']}.device_location_uplevelled"

# Convert integer dates to datetime objects
start_date_dt = datetime.strptime(str(start_date), '%Y%m%d')
end_date_dt = datetime.strptime(str(end_date), '%Y%m%d')

# Function to insert data with retry mechanism
def insert_data_with_retry(df, table_name, con, retries=3, delay=5):
    for attempt in range(retries):
        try:
            df.to_sql(
                table_name, 
                con, 
                index=False, 
                if_exists="append", 
                method="multi"
            )
            logging.info(f"Inserted data into table {table_name}")
            break
        except Exception as e:
            logging.error(f"Attempt {attempt+1} failed with error: {e}")
            if attempt < retries - 1:
                time.sleep(delay)
            else:
                logging.error(f"Failed to insert data into table {table_name} after {retries} attempts")

# Loop through each day from start_date to end_date
current_date = start_date_dt
while current_date <= end_date_dt:
    # Calculate the lookback and lookahead dates
    lookback_date = current_date - timedelta(days=1)
    lookahead_date = current_date + timedelta(days=35)
    
    # Format dates for the SQL query
    formatted_lookback_date = lookback_date.strftime('%Y%m%d')
    formatted_current_date = current_date.strftime('%Y%m%d')
    formatted_lookahead_date = lookahead_date.strftime('%Y%m%d')
    
    # Construct the SQL query
    query = f"""
    SELECT 
        cuebiq_id, 
        event_zoned_datetime, 
        processing_date,
        lat,
        lng
    FROM {pe_dl_table}
    WHERE 
        processing_date BETWEEN {formatted_lookback_date} AND {formatted_lookahead_date}
        AND country_code = '{country_code}' 
        AND event_zoned_datetime IS NOT NULL
        AND TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s')) IS NOT NULL
        AND DATE(TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s'))) = date_parse('{formatted_current_date}', '%Y%m%d')
    """
    
    logging.info(f"Executing SQL query for date {formatted_current_date}: {query}")
    
    try:
        pe_dl_table_gen = sql_engine.read_sql_chunked(query)
        
        # Convert the generator to a DataFrame
        chunks = [chunk for chunk in pe_dl_table_gen]
        if chunks:
            pe_dl_table_df = pd.concat(chunks, ignore_index=True)
            
            # Calculate geohashes
            pe_dl_table_df['geohash5'] = pe_dl_table_df.apply(lambda row: geohash2.encode(row['lat'], row['lng'], precision=5), axis=1)
            pe_dl_table_df['geohash3'] = pe_dl_table_df.apply(lambda row: geohash2.encode(row['lat'], row['lng'], precision=3), axis=1)
            
            # Aggregate data for geohash5
            aggregated_data_5 = pe_dl_table_df.groupby('geohash5').agg(
                no_of_points=('geohash5', 'size'),
                no_of_unique_users=('cuebiq_id', 'nunique')
            ).reset_index()
            
            # Insert aggregated data for geohash5 into SQL table
            table_name_agg5 = f"pd_{country_code}_{formatted_current_date}_agg5"
            insert_data_with_retry(aggregated_data_5, table_name_agg5, con)
            
            # Aggregate data for geohash3
            aggregated_data_3 = pe_dl_table_df.groupby('geohash3').agg(
                no_of_points=('geohash3', 'size'),
                no_of_unique_users=('cuebiq_id', 'nunique')
            ).reset_index()
            
            # Insert aggregated data for geohash3 into SQL table
            table_name_agg3 = f"pd_{country_code}_{formatted_current_date}_agg3"
            insert_data_with_retry(aggregated_data_3, table_name_agg3, con)
        
    except Exception as e:
        logging.error(f"Error while processing data for date {formatted_current_date}: {e}")
    
    # Move to the next day
    current_date += timedelta(days=1)

logging.info("Data extraction, aggregation, and saving completed.")

end_time = time.time()  # End timing

# Calculate and print the total time taken
total_time = end_time - start_time
logging.info(f"Total time taken: {total_time} seconds")


2024-06-04 19:34:15,684 - INFO - Executing SQL query for date 20190101: 
    SELECT 
        cuebiq_id, 
        event_zoned_datetime, 
        processing_date,
        lat,
        lng
    FROM cuebiq.paas_cda_pe_v3.device_location_uplevelled
    WHERE 
        processing_date BETWEEN 20181231 AND 20190205
        AND country_code = 'IN' 
        AND event_zoned_datetime IS NOT NULL
        AND TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s')) IS NOT NULL
        AND DATE(TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s'))) = date_parse('20190101', '%Y%m%d')
    
/tmp/ipykernel_723/3963072833.py:36: UserWarning: The provided table name 'pd_IN_20190101_agg5' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  df.to_sql(
2024-06-04 19:49:26,410 - INFO - Inserted data into table pd_IN_20190101_agg5
/tmp/ipykernel_723/3963072833.py:36: UserWarni

KeyboardInterrupt: 